# 一、准备工作

- 导入工作路径和需要的库，检查所有需要的目录

## 1.1 库

In [1]:
from utils.imports import *
from utils.functions import *
from utils.paths import *

%matplotlib inline


dropout_rate = 0.35
width = 32

Using TensorFlow backend.


## 1.2 目录

In [ ]:
dir_check()

## 1.3 函数

# 二、处理原始数据

- 生成数据的原始npy、有结节肺切片npy、结节掩膜npy，提取origin、spacing至csv

### 2.1 处理Train

In [ ]:
PATH = {'annotations_path' : annotations_train_path,
        'src' : src_train,
        'dst_nodules' : mask_train,
        'dst_full_lungs' : full_lung_train}

In [ ]:
patients = load_train(PATH)
df_node = pd.read_csv(PATH['annotations_path']+"annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
    
df_node = df_add_column(df_node)
df_node = df_node.dropna()

In [ ]:
r = Parallel(n_jobs=-1)(delayed(create_masks_for_patient_watershed)(patient,df_node,PATH) for patient in tqdm(sorted(patients)))
i = zip(*r)

In [ ]:
save_to_csv(i,df_node,PATH)

### 2.2 处理Val

#### 处理结果保存至train

In [ ]:
PATH = {'annotations_path' : annotations_val_path,
        'src' : src_val,
        'dst_nodules' : mask_train,
        'dst_full_lungs' : full_lung_val}

In [ ]:
patients = load_train(PATH)
df_node = pd.read_csv(PATH['annotations_path']+"annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(patients, file_name))

df_node = df_add_column(df_node)
df_node = df_node.dropna()

In [ ]:
r = Parallel(n_jobs=-1)(delayed(create_masks_for_patient_watershed)(patient,df_node,PATH) for patient in tqdm(sorted(patients)))
i = zip(*r)

In [ ]:
save_to_csv(i,df_node,PATH)

# 三、训练2D分割模型

In [2]:
PATH = {'lung_path' : mask_train +  'lung_mask/',
        'nodule_path' : mask_train + 'nodule_mask/',
        'model_path' : model_path,
        'batch_size' : 1,
        'epochs' : 50,
        'dropout_rate' : dropout_rate,
        'width' : width}

In [3]:
t1 = int(len(sorted([x for x in os.listdir(PATH['lung_path']) if x !='.DS_Store']))*0.8*0.1)
t2 = int(len(sorted([x for x in os.listdir(PATH['lung_path']) if x !='.DS_Store']))*0.1)
PATH['t'] = [0,t1,t2]

In [ ]:
if_old = True #设置True,继续训练模型,否则重新训练模型

if if_old:
    model = unet_fit('FenGe531', unet_model,PATH, load_check = True, check_name = 'FenGe531')
else:
    model = unet_fit('FenGe531_64', unet_model,PATH, load_check = False)

Number of 2D slices in CT image: 902
Number of 2D slices in CT image: 251
('Training samples:', 902, 'Validation samples:', 125)
Epoch 1/50
902/902 [==============================] - 319s - loss: -1.4380 - dice_coef: 1.4380 - val_loss: -0.4655 - val_dice_coef: 0.4655

# 四、分割数据

- 分割Test、Train，获得所有预测结节的坐标和直径，并转换成世界坐标

### 4.1 生成test数据的原始npy，提取origin、spacing至csv

In [ ]:
PATH = {'annotations_path' : annotations_test_path,
        'src' : src_test,
        'dst_nodules' : mask_test,
        'dst_full_lungs' : full_lung_test,
        'model_path' : model_path,
        
        'dropout_rate' : dropout_rate,
        'width' : width,}

In [ ]:
patients = load_train(PATH)

In [ ]:
r = Parallel(n_jobs=-1)(delayed(create_masks_for_test)(patient,PATH) for patient in tqdm(sorted(patients)))
i = zip(*r)

In [ ]:
save_test_full_to_csv(i,PATH)

### 4.2 使用模型分割Test数据

In [ ]:
PATH = {'annotations_path' : annotations_test_path,
        'npy_file' : full_lung_test,
        
        
        'dropout_rate' : dropout_rate,
        'model_path' : model_path,
        'width' : width,}

In [ ]:
file_list = get_dirfiles(PATH['npy_file'])

In [ ]:
pred = pred_result(file_list, PATH)

In [ ]:
save_pred_to_csv(pred,PATH)

### 4.3 提取test质心

In [ ]:
PATH = {'annotations_path' : annotations_test_path}

In [ ]:
final = merge_pred_origin_csv(PATH)

In [ ]:
result = result_v2w(final,PATH)
result.to_csv(PATH['annotations_path']+'annotations_final2cls.csv',index=None)

### 4.4 使用模型分割Val数据

In [ ]:
PATH = {'annotations_path' : annotations_val_path,
        'src' : src_val,
        'dst_nodules' : mask_val,
        'dst_full_lungs' : full_lung_val,
        'model_path' : model_path,
        
        'dropout_rate' : dropout_rate,
        'width' : width,}

In [ ]:
patients = load_train(PATH)

In [ ]:
r = Parallel(n_jobs=-1)(delayed(create_masks_for_test)(patient,PATH) for patient in tqdm(sorted(patients)))
i = zip(*r)

In [ ]:
save_test_full_to_csv(i,PATH)

### x

In [ ]:
PATH = {'annotations_path' : annotations_val_path,
        'npy_file' : full_lung_val,
        
        
        'dropout_rate' : dropout_rate,
        'model_path' : model_path,
        'width' : width,}

In [ ]:
file_list = get_dirfiles(PATH['npy_file'])

In [ ]:
pred = pred_result(file_list, PATH)

In [ ]:
save_pred_to_csv(pred,PATH)

### 4.5 提取Val质心

In [ ]:
PATH = {'annotations_path' : annotations_val_path}

In [ ]:
final = merge_pred_origin_csv(PATH)

In [ ]:
result = result_v2w(final,PATH)
result.to_csv(PATH['annotations_path']+'annotations_final2cls.csv',index=None)

# 五、数据验证分割覆盖率

- 将Train的真实结节设置为正样本，其他为假样本。
- 顺便可以看下分割模型的覆盖率

## Val

- 预测结果与给定标注区域相交比例：51.67%
- 预测质心在给定标注区域内的比例：46.84%
- 预测质心在给定标注直径64内比例：79.18%
- 过拟合

In [ ]:
PATH = {'annotations_path' : annotations_val_path}

In [ ]:
ifin(PATH)

# 六、训练分类模型（2D）

## 6.1 生成2D分类模型的Train样本

In [ ]:
PATH = {'npy_file' : full_lung_train,
        'annotations_path' : annotations_train_path,
        'pics_path_true' : train_true_pics_path_2d,
        'pics_path_false' : train_false_pics_path_2d}

In [ ]:
full = pd.read_csv(PATH['annotations_path'] + "annotations_final2cls.csv", index_col=None)
file_list = get_dirfiles(PATH['npy_file'])
generate_cls_sample2d(file_list,full,PATH)

## 6.2 生成2D分类模型的Val样本

In [ ]:
PATH = {'npy_file' : full_lung_val,
        'annotations_path' : annotations_val_path,
        'pics_path_true' : val_true_pics_path_2d,
        'pics_path_false' : val_false_pics_path_2d}

In [ ]:
full = pd.read_csv(PATH['annotations_path'] + "annotations_final2cls.csv", index_col=None)
file_list = get_dirfiles(PATH['npy_file'])
generate_cls_sample2d(file_list,full,PATH)

## 6.3 生成2D分类模型的Test样本

In [ ]:
PATH = {'npy_file' : full_lung_test,
        'annotations_path' : annotations_test_path,
        'pics_path_true' : test_true_pics_path_2d,
        'pics_path_false' : test_false_pics_path_2d}

In [ ]:
full = pd.read_csv(PATH['annotations_path'] + "annotations_final2cls.csv", index_col=None)
file_list = get_dirfiles(PATH['npy_file'])
generate_cls_sample2d(file_list,full,PATH,train=False)

# 七、分类模型

In [ ]:
PATH = {'pics_path_true' : train_true_pics_path,
        'pics_path_false' : train_false_pics_path,
        'model_path' : model_path,
        'batch_size' : 2,
        'epochs' : 5,
        'dropout_rate' : 0.2,
        'width' : width,
        't' : [0,500,200]}

In [ ]:
#if_run = False #设置是否要训练模型
if_run = True
if if_run:
    
    model = fenlei_fit('FenLei', fenlei_model,PATH, load_check = True, check_name = 'FenLei')